In [ ]:
import os
import numpy as np
import logging
import tensorflow as tf
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from sklearn.metrics import precision_score, f1_score, confusion_matrix
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback, LearningRateScheduler

# Set the image size and other constants
IMG_HEIGHT = 512
IMG_WIDTH = 512
NUM_CLASSES = 1  # Single class for binary segmentation

# Suppress TensorFlow warnings and error messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

def load_dataset(image_dir, mask_dir, img_size):
    images = []
    masks = []
    
    for image_name in tqdm(os.listdir(image_dir)):
        if image_name.lower().endswith(('.jpeg', '.jpg')):
            base_name = os.path.splitext(image_name)[0]
            mask_name = f"{base_name}_mask.jpg"
            
            img_path = os.path.join(image_dir, image_name)
            mask_path = os.path.join(mask_dir, mask_name)
            
            if os.path.exists(img_path) and os.path.exists(mask_path):
                try:
                    image = load_img(img_path, target_size=img_size)
                    mask = load_img(mask_path, target_size=img_size, color_mode="grayscale")
                    
                    image = img_to_array(image)
                    mask = img_to_array(mask)
                    
                    images.append(image)
                    masks.append(mask)
                except Exception as e:
                    print(f"Error processing {image_name}: {e}")
            else:
                print(f"Mask not found for image: {image_name}")
    
    images = np.array(images, dtype=np.float32) / 255.0
    masks = np.array(masks, dtype=np.float32) / 255.0

    masks = (masks > 0.5).astype(np.uint8)
    
    return images, masks

# Define directories
train_image_dir = '/home/syam.varnatt/THESIS/images/train/images'
train_mask_dir = '/home/syam.varnatt/THESIS/images/train/masks'
val_image_dir = '/home/syam.varnatt/THESIS/images/val/images'
val_mask_dir = '/home/syam.varnatt/THESIS/images/val/masks'
SAVE_DIRECTORY = '/home/syam.varnatt/THESIS/comparison-model/deeplab/normal/results'
os.makedirs(SAVE_DIRECTORY, exist_ok=True)

# Load datasets
train_images, train_masks = load_dataset(train_image_dir, train_mask_dir, (IMG_HEIGHT, IMG_WIDTH))
print("Training dataset loaded completely")
val_images, val_masks = load_dataset(val_image_dir, val_mask_dir, (IMG_HEIGHT, IMG_WIDTH))
print("Validation dataset loaded completely")


In [ ]:
# DeepLabV3+ architecture 
class ResizeLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(ResizeLayer, self).__init__(**kwargs)

    def call(self, inputs):
        source, target = inputs
        target_size = tf.shape(target)[1:3]
        return tf.image.resize(source, target_size)

def aspp_block(x, num_filters):
    # ASPP branch 1: 1x1 convolution
    x1 = tf.keras.layers.Conv2D(num_filters, (1, 1), padding='same', use_bias=False)(x)
    x1 = tf.keras.layers.BatchNormalization()(x1)
    x1 = tf.keras.layers.ReLU()(x1)

    # ASPP branch 2: 3x3 convolution with dilation rate 6
    x2 = tf.keras.layers.Conv2D(num_filters, (3, 3), dilation_rate=6, padding='same', use_bias=False)(x)
    x2 = tf.keras.layers.BatchNormalization()(x2)
    x2 = tf.keras.layers.ReLU()(x2)

    # ASPP branch 3: Global average pooling followed by 1x1 convolution
    x3 = tf.keras.layers.GlobalAveragePooling2D()(x)
    x3 = tf.keras.layers.Reshape((1, 1, x.shape[-1]))(x3)
    x3 = tf.keras.layers.Conv2D(num_filters, (1, 1), padding='same', use_bias=False)(x3)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    x3 = tf.keras.layers.ReLU()(x3)

    # Resize the pooled features to match the size of other branches using a custom layer
    x3 = ResizeLayer()([x3, x1])

    # Concatenate all the branches
    x = tf.keras.layers.Concatenate()([x1, x2, x3])
    return x

def DeeplabV3Plus(image_size, num_classes):
    base_model = tf.keras.applications.MobileNetV2(
        weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3)
    )

    # Extract feature map from the last block of MobileNetV2
    x = base_model.get_layer('block_13_expand_relu').output

    # Apply simplified ASPP block
    x = aspp_block(x, 64)  # Reduced number of filters

    # Dynamically upsample the output to match the original input size / 4 using a custom layer
    x = ResizeLayer()([x, base_model.input])

    # Extract low-level features from an earlier layer of MobileNetV2
    low_level_feature = base_model.get_layer('block_3_expand_relu').output
    low_level_feature = tf.keras.layers.Conv2D(16, (1, 1), padding='same', use_bias=False)(low_level_feature)  # Reduced number of filters
    low_level_feature = tf.keras.layers.BatchNormalization()(low_level_feature)
    low_level_feature = tf.keras.layers.ReLU()(low_level_feature)

    # Resize the low-level features to match the size of the high-level features using a custom layer
    low_level_feature = ResizeLayer()([low_level_feature, x])

    # Concatenate the high-level and low-level features
    x = tf.keras.layers.Concatenate()([x, low_level_feature])

    # Further processing with a few convolutional layers
    x = tf.keras.layers.Conv2D(64, (3, 3), padding='same', use_bias=False)(x)  # Reduced number of filters
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    # Apply dropout
    #x = tf.keras.layers.Dropout(0.2)(x)  # Reduced dropout rate

    # Final 1x1 convolution to produce the output segmentation map
    x = tf.keras.layers.Conv2D(num_classes, (1, 1), padding='same')(x)
    x = tf.keras.layers.Activation('sigmoid')(x)  # Add sigmoid activation for binary segmentation

    # Final upsampling to match the input image size using a custom layer
    x = ResizeLayer()([x, base_model.input])

    # Define the model
    return tf.keras.Model(inputs=base_model.input, outputs=x)

# Enable mixed precision
#tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Instantiate the DeepLabV3+ model
model = DeeplabV3Plus((IMG_HEIGHT, IMG_WIDTH), NUM_CLASSES)

# Learning Rate Scheduler
#def lr_schedule(epoch, lr):
 #   return lr * 0.9  # Reduce learning rate by 10% every epoch

#lr_scheduler = LearningRateScheduler(lr_schedule)

# Compile the model with appropriate loss and metrics
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
class VisualizationCallback(tf.keras.callbacks.Callback):
    def __init__(self, val_images, val_masks, save_dir, num_samples=3):
        super(VisualizationCallback, self).__init__()
        self.val_images = val_images
        self.val_masks = val_masks
        self.save_dir = save_dir
        self.num_samples = num_samples  # Number of samples to visualize per epoch
        os.makedirs(save_dir, exist_ok=True)  # Ensure the save directory exists

    def on_epoch_end(self, epoch, logs=None):
        # Randomly select a few samples to visualize predictions
        idxs = np.random.choice(len(self.val_images), self.num_samples, replace=False)
        images_to_show = self.val_images[idxs]
        masks_to_show = self.val_masks[idxs]
        
        # Use the model attribute directly, which is set automatically by Keras
        predictions = self.model.predict(images_to_show)

        # Plot and save the images, masks, and predictions
        plt.figure(figsize=(15, 10))
        for i in range(self.num_samples):
            plt.subplot(self.num_samples, 3, i * 3 + 1)
            plt.imshow(images_to_show[i])
            plt.title(f'Input Image {i+1}')
            plt.axis('off')

            plt.subplot(self.num_samples, 3, i * 3 + 2)
            plt.imshow(masks_to_show[i].squeeze(), cmap='gray')
            plt.title(f'True Mask {i+1}')
            plt.axis('off')

            plt.subplot(self.num_samples, 3, i * 3 + 3)
            plt.imshow(predictions[i].squeeze(), cmap='gray')
            plt.title(f'Predicted Mask {i+1}')
            plt.axis('off')

        plt.tight_layout()
        # Save the plot to file
        plt.savefig(os.path.join(self.save_dir, f'epoch_{epoch+1:02d}_predictions.png'))
        plt.close()
        print(f"Saved visualization for epoch {epoch+1}.")

# Instantiate the visualization callback
visualization_callback = VisualizationCallback(val_images, val_masks, SAVE_DIRECTORY)


In [ ]:
import time
class TrainingTimeCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.start_time = time.time()

    def on_train_end(self, logs=None):
        end_time = time.time()
        training_time = end_time - self.start_time
        print(f"Training Time: {training_time / 60:.2f} minutes")

# Create an instance of the callback
training_time_callback = TrainingTimeCallback()

checkpoint = ModelCheckpoint('deeplab_model.keras', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

# Train the model
history = model.fit(
    train_images, train_masks,
    validation_data=(val_images, val_masks),
    batch_size=8,
    epochs=100,
    callbacks=[checkpoint, early_stopping, training_time_callback, visualization_callback],
    verbose=1 
)
print("Training completed.")

# Save the history to an Excel file
history_df = pd.DataFrame(history.history)
history_df.to_excel(os.path.join("training_history.xlsx"))

In [ ]:

# Predict on validation set
val_preds = model.predict(val_images)

# Convert predictions to binary
val_preds_bin = (val_preds > 0.3).astype(int)

# Calculate additional metrics
precision = precision_score(val_masks.flatten(), val_preds_bin.flatten())
f1 = f1_score(val_masks.flatten(), val_preds_bin.flatten())

# Calculate Mean IoU
mean_iou_metric = tf.keras.metrics.MeanIoU(num_classes=2) 
mean_iou_metric.update_state(val_masks, val_preds_bin)
mean_iou = mean_iou_metric.result().numpy()

# Save metrics to Excel
metrics_df = pd.DataFrame({
    'Precision': [precision],
    'F1 Score': [f1],
    'Mean IoU': [mean_iou]
})
print(f"Precision: ", precision, "F1 score: ", f1, "Mean Iou : ", mean_iou)

# Plot training history
history_df = pd.read_excel('training_history.xlsx')

plt.figure()
plt.plot(history_df['loss'], label='Training Loss')
plt.plot(history_df['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('loss_plot.png')
plt.show()

plt.figure()
plt.plot(history_df['accuracy'], label='Training Accuracy')
plt.plot(history_df['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('accuracy_plot.png')
plt.show()

# Save confusion matrix as heatmap
cm = confusion_matrix(val_masks.flatten(), val_preds_bin.flatten())
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.png')
plt.show()

# Visualize some predictions on validation set
for i in range(3):
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    axs[0].imshow(val_images[i])
    axs[0].set_title('Input Image')
    axs[1].imshow(val_masks[i].squeeze(), cmap='gray')
    axs[1].set_title('Ground Truth')
    axs[2].imshow(val_preds_bin[i].squeeze(), cmap='gray')
    axs[2].set_title('Predicted Mask')
    plt.savefig(f'prediction_{i}.png')
    plt.show()

In [ ]:
#calculate inference time

# Use a small batch from validation data
sample_images = val_images[:5]

# Measure inference time
start_time = time.time()
predictions = model.predict(sample_images)
end_time = time.time()

inference_time = (end_time - start_time) / len(sample_images)
print(f"Average Inference Time per Image: {inference_time:.4f} seconds")

In [ ]:

import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.keras.models import load_model

# Define input size
input_size = (512, 512, 3)

# Function to preprocess images
def preprocess_image(image_path, target_size):
    # Load image
    image = load_img(image_path, target_size=target_size[:2])
    # Convert to numpy array
    image_array = img_to_array(image)
    # Normalize the image
    image_array = image_array / 255.0
    return image_array

# Function to save images
def save_image(image_array, save_path):
    # Convert array to image
    image = array_to_img(image_array)
    # Save the image
    image.save(save_path)

# Function to display images
def display_images(images, titles=None, cmap='gray'):
    plt.figure(figsize=(20, 10))
    for i, image in enumerate(images):
        plt.subplot(3, 4, i + 1)  # Adjust depending on the number of images
        plt.imshow(image, cmap=cmap)
        if titles:
            plt.title(titles[i])
        plt.axis('off')
    plt.show()

# Load and preprocess images
image_folder = '/home/syam.varnatt/THESIS/Plot_Images(x10,x20)'
image_paths = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder) if fname.endswith('.png')]

# Sort the image paths by filenames
image_paths = sorted(image_paths)

# Select a subset of images
image_paths = image_paths[0:]

# Predict segmentation masks for each image
# Extract image names from paths
image_names = [os.path.basename(path) for path in image_paths]

# Define the folder path to save images and masks
save_folder = '/home/syam.varnatt/THESIS/comparison-model/deeplab/normal/predict_height'
os.makedirs(save_folder, exist_ok=True)

# Predict segmentation masks and save images and masks
predicted_masks = []
for i, image_path in enumerate(image_paths):
    # Preprocess image
    image = preprocess_image(image_path, input_size)
    image_batch = np.expand_dims(image, axis=0)
    # Predict mask
    predicted_mask = model.predict(image_batch)[0]
    predicted_mask = (predicted_mask > 0.5).astype(np.uint8)
    
    # Save the original image
    original_image = img_to_array(load_img(image_path, target_size=input_size[:2])) / 255.0
    save_image(original_image, os.path.join(save_folder, f'original_{image_names[i]}'))
    
    # Save the predicted mask
    save_image(predicted_mask, os.path.join(save_folder, f'mask_{image_names[i]}'))
    
    predicted_masks.append(predicted_mask)

In [ ]:
#calculate inference time for predicting heights

# Use a small batch from validation data
sample_images = image_batch[:5]

# Measure inference time
start_time = time.time()
predictions = model.predict(sample_images)
end_time = time.time()

inference_time = (end_time - start_time) / len(sample_images)
print(f"Average Inference Time per flight height predicted Image: {inference_time:.4f} seconds")